# Quantum Algorithm Speed-Ups
An overview of speed-ups from [Quantum Algorithm Zoo](https://quantumalgorithmzoo.org/).

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


url = "https://quantumalgorithmzoo.org/"
response = requests.get(url)
content = response.content
parsed_content = BeautifulSoup(content, "html.parser")


def extract_algorithms_and_speedups(parsed):
    data = []
    current_section = None
    for element in parsed.find_all(["h2", "b"]):
        if element.name == "h2":
            current_section = element.get_text(strip=True)
        elif element.name == "b" and element.get_text(strip=True) == "Algorithm:":
            algorithm = element.find_next_sibling(string=True).strip()
            speedup_element = element.find_next("b", string="Speedup:")
            if speedup_element:
                speedup = speedup_element.find_next_sibling(string=True).strip()
                if current_section:
                    data.append([current_section, algorithm, speedup])
    return data


# Extract relevant data from the URL
algorithms_and_speedups = extract_algorithms_and_speedups(parsed_content)

In [8]:
speedups = pd.DataFrame(
    algorithms_and_speedups, columns=["category", "algorithm", "speedup"]
)

# Clean up and simplification
speedup_mapping = {
    "Constant factor": "constant",
    "Exponential": "exponential",
    "Exponential over P, none over BPP": "exponential",
    "Polynomial": "polynomial",
    "Polynomial (quartic)": "quartic",
    "Polynomial Directly, Superpolynomial Recursively": "polynomial",
    "Superpolynomial": "superpolynomial",
    "Varies": "variable",
    "Various": "variable",
}

speedups["speedup"] = speedups["speedup"].replace(speedup_mapping)

## Distribution of Speed-Ups

In [9]:
counts_by_speedup = speedups.groupby("speedup").size()
((counts_by_speedup / counts_by_speedup.sum()) * 100).round(1).sort_values(
    ascending=False
)

speedup
superpolynomial    45.7
polynomial         42.9
variable            5.7
exponential         2.9
constant            1.4
quartic             1.4
dtype: float64

In [10]:
counts_by_category = speedups.groupby("category").size()
((counts_by_category / counts_by_category.sum()) * 100).round(1).sort_values(
    ascending=False
)

category
Oracular Algorithms                             44.3
Algebraic and Number Theoretic Algorithms       20.0
Optimization, Numerics, and Machine Learning    18.6
Approximation and Simulation Algorithms         17.1
dtype: float64

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", palette="colorblind")

plt.figure(figsize=(6, 4))
ax = sns

speedups["Speedup"] = pd.to_numeric(
    speedups["Speedup"].str.replace(",", ""), errors="coerce"
)

speedups = speedups.dropna(subset=["Speedup"])

plt.figure(figsize=(12, 6))
sns.boxplot(data=speedups, x="Section", y="Speedup")
plt.xticks(rotation=90)
plt.title("Distribution of Speedups by Section")
plt.tight_layout()
plt.show()

speedup_counts = speedups["Section"].value_counts()

# Plot the counts of speedups across sections
plt.figure(figsize=(12, 6))
sns.barplot(x=speedup_counts.index, y=speedup_counts.values)
plt.xticks(rotation=90)
plt.title("Counts of Speedups Across Sections")
plt.xlabel("Section")
plt.ylabel("Count")
plt.tight_layout()
plt.show()